In [10]:
import pandas as pd
import gc
import os

In [12]:
# parquet 파일 데이터를 읽어온다.
df1 = pd.read_parquet('open/train/1.회원정보/201807_train_회원정보.parquet')
df1

,기준년월,ID,남녀구분코드,연령,Segment,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,...,할인금액_제휴연회비_B0M,청구금액_기본연회비_B0M,청구금액_제휴연회비_B0M,상품관련면제카드수_B0M,임직원면제카드수_B0M,우수회원면제카드수_B0M,기타면제카드수_B0M,카드신청건수,Life_Stage,최종카드발급경과월
0,201807,TRAIN_000000,2,40대,D,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),22
1,201807,TRAIN_000001,1,30대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),18
2,201807,TRAIN_000002,1,30대,C,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,20
3,201807,TRAIN_000003,2,40대,D,1,1,0,1,2,...,0,0,0,0개,0개,0개,0개,1,자녀성장(2),17
4,201807,TRAIN_000004,2,40대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,1,자녀성장(1),15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,201807,TRAIN_399995,2,70대이상,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,노년생활,34
399996,201807,TRAIN_399996,2,50대,D,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(2),19
399997,201807,TRAIN_399997,1,30대,C,1,1,0,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀출산기,13
399998,201807,TRAIN_399998,1,40대,E,1,1,1,1,1,...,0,0,0,0개,0개,0개,0개,0,자녀성장(1),22


In [7]:
# del df1
# del df2
# gc.collect()

0

- 하나의 파일에서 작업을 다 하려고 하지 마세요
- 데이터를 읽어온 다음 데이터를 가공했다면 가공한 데이터를 파일로 저장하고 새로운 ipynb 파일을 만들어서 저장한 데이터를 읽어와서 작업하는 형태로 해주세요
- 머신러닝은 xgboost만 써도됨

### 스크립트 병합

In [27]:
import pandas as pd
import os

# ✅ 루트 폴더 경로
base_folder = r'C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open'

# ✅ 대상 월
months = ["07", "08", "09", "10", "11", "12"]

# ✅ train, test 폴더
data_types = ["train", "test"]

# ✅ 실제 하위 폴더 이름 목록 (예: '1.회원정보' → 카테고리명 추출)
category_folder_names = [
    "1.회원정보", "2.신용정보", "3.승인매출정보", "4.청구입금정보",
    "5.잔액정보", "6.채널정보", "7.마케팅정보", "8.성과정보"
]

def merge_monthly_data(data_type, folder_name):
    # 예: '1.회원정보' → '회원정보'만 추출
    category = folder_name.split('.')[-1]
    merged_list = []

    # 경로 설정: C:\...\train\1.회원정보
    category_folder_path = os.path.join(base_folder, data_type, folder_name)

    for month in months:
        filename = f"2018{month}_{data_type}_{category}.parquet"
        file_path = os.path.join(category_folder_path, filename)

        if os.path.exists(file_path):
            try:
                df = pd.read_parquet(file_path, engine="pyarrow")
                merged_list.append(df)
                print(f"✅ {file_path} 불러오기 완료")
            except Exception as e:
                print(f"❌ 읽기 오류: {file_path} → {e}")
        else:
            print(f"⚠️ 파일 없음: {file_path}")

    if merged_list:
        # 파일 병합
        merged_df = pd.concat(merged_list, ignore_index=True)

        # CSV 저장 경로 설정
        output_path = os.path.join(base_folder, data_type, f"2018_{data_type}_{category}.csv")

        # ✅ 한글 인코딩 깨짐 방지
        merged_df.to_csv(output_path, index=False, encoding='utf-8-sig')

        print(f"✅ 저장 완료: {output_path} (Shape: {merged_df.shape})")
    else:
        print(f"❌ {data_type}_{category} 데이터 없음")

# ✅ 전체 데이터 병합 실행
for data_type in data_types:
    for folder_name in category_folder_names:
        merge_monthly_data(data_type, folder_name)


✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\1.회원정보\201807_train_회원정보.parquet 불러오기 완료
✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\1.회원정보\201808_train_회원정보.parquet 불러오기 완료
✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\1.회원정보\201809_train_회원정보.parquet 불러오기 완료
✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\1.회원정보\201810_train_회원정보.parquet 불러오기 완료
✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\1.회원정보\201811_train_회원정보.parquet 불러오기 완료
✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\1.회원정보\201812_train_회원정보.parquet 불러오기 완료
✅ 저장 완료: C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\2018_train_회원정보.csv (Shape: (2400000, 78))
✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\2.신용정보\201807_train_신용정보.parquet 불러오기 완료
✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\2.신용정보\201808_train_신용정보.parquet 불러오기 완료
✅ C:\Users\KING\Desktop\workspace\11_파이널프로젝트관련\open\train\2.신용정보\201809_train_신용정보.parquet 불러오기 완료
✅